In [47]:
import pandas as pd
import numpy as np
from itertools import product
import os
import matplotlib.pyplot as plt
%matplotlib inline 

In [76]:
# WIP: creating mean encodings as features

In [7]:
transactions = pd.read_csv('sales_train.csv.gz')
items = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')

print(transactions.shape)
print(items.shape)
print(item_categories.shape)
print(shops.shape)
transactions.head(3)

(2935849, 6)
(22170, 3)
(84, 2)
(60, 2)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
1,03.01.2013,0,25,2552,899.0,1.0
2,05.01.2013,0,25,2552,899.0,-1.0


In [8]:
transactions[['Day','Month','Year']]=transactions.date.str.split('.',expand=True)
train = transactions.copy()
del train['date']

In [38]:
train.head(5)

,date_block_num,shop_id,item_id,item_price,item_cnt_day,Day,Month,Year
0,0,59,22154,999.00,1.0,02,01,2013
1,0,25,2552,899.00,1.0,03,01,2013
2,0,25,2552,899.00,-1.0,05,01,2013
3,0,25,2554,1709.05,1.0,06,01,2013
4,0,25,2555,1099.00,1.0,15,01,2013


In [48]:
sales = transactions
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

# Turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# Get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

# Fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
# Join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
# Sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

/usr/local/lib/python3.5/dist-packages/pandas/core/groupby.py:4189: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [49]:
all_data.head()

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0


In [69]:
target_mean = all_data.groupby('item_id').target.mean()
target_mean.head()

item_id
0    0.020000
1    0.023810
2    0.019802
3    0.019802
4    0.020000
Name: target, dtype: float64

In [70]:
all_data['item_id_'+ 'mean_target'] =  all_data['item_id'].map(target_mean)


In [72]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_id_mean_target
139255,0,19,0,0.0,0.022222
141495,0,27,0,0.0,0.056834
144968,0,28,0,0.0,0.141176
142661,0,29,0,0.0,0.037383
138947,0,32,0,6.0,1.319042


In [75]:
all_data[all_data.date_block_num==33].head()

,shop_id,item_id,date_block_num,target,item_id_mean_target
10864839,2,30,33,0.0,1.355613
10860073,2,31,33,1.0,0.931213
10861046,2,32,33,0.0,1.319042
10859871,2,33,33,0.0,0.527112
10864840,2,40,33,0.0,0.070943
